**Test de k-means sur séries temporelles**



In [1]:
#Importation des modules nécessaires
from scipy import spatial
import numpy as np
import math
import fastdtw as dtw
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine
from cookie_clusters import *

Ici on travaille sur des données multivariées, donc une liste de séries temporelles, avec chaque série temporelle composée de T vecteurs de 4 valeurs (=4 bandes pour chaque pixel).
T = nombre d'élements dans chaque série temporelles (donc nombre d'images)
Le nombre de séries temporelles correspond au nombre de pixels sur une image.

In [2]:
# Regrouper les séquences dans une liste
data = [[[1,2,3,5],[10,23,4,8],[5,8,9,2],[4,2,12,4]], [[1,8,3,7],[9,11,14,21],[5,5,7,8],[4,3,3,5]], [[2,1,2,10],[3,10,3,7],[2,5,8,19],[1,4,9,20]]]
#data = [sequence1,sequence2,sequence3,sequence4]
data=np.array(data)
#data=data.reshape((4,6,1))

print("Nombre de pixels dans chaque image:",data.shape[0])
print("Nombre d'images:",data.shape[1])
print("Nombre de bandes pour chaque pixel:",data.shape[2])


Nombre de pixels dans chaque image: 3
Nombre d'images: 4
Nombre de bandes pour chaque pixel: 4


In [3]:
#mesure de similarité dtw entre deux séries temporelles. x = un pixel sur un intervalle de temps
def dtw_euclidien(x, x_prime):
  R=np.zeros((len(x),len(x_prime)))
  for i in range(len(x)):
    for j in range(len(x_prime)):
      R[i, j] = spatial.distance.euclidean(x[i], x_prime[j]) ** 2
      if i > 0 or j > 0:
        R[i, j] += min(
          R[i-1, j  ] if i > 0             else math.inf,
          R[i  , j-1] if j > 0             else math.inf,
          R[i-1, j-1] if (i > 0 and j > 0) else math.inf
        )

  return R[-1, -1] ** (1/2)

#pour tester avec le cosin
def dtw_cosine(x, x_prime):
  R=np.zeros((x.shape[0],x_prime.shape[0]))
  for i in range(len(x)):
    for j in range(len(x_prime)):
      R[i, j] = spatial.distance.cosine(x[i], x_prime[j])
      if i > 0 or j > 0:
        R[i, j] += min(
          R[i-1, j  ] if i > 0             else math.inf,
          R[i  , j-1] if j > 0             else math.inf,
          R[i-1, j-1] if (i > 0 and j > 0) else math.inf
        )

  return R[-1, -1]



print("dtw euclidien que j'ai fait:",dtw_euclidien(data[1],data[2]))
print("fastdtw euclidien:",dtw.fastdtw(data[1],data[2],dist=euclidean)[0])
print("dtw cosine que j'ai fait:",dtw_cosine(data[1],data[2]))
print("fastdtw cosine:",dtw.fastdtw(data[1],data[2],dist=cosine)[0])

#scipy.spatial.distance.cdist


dtw euclidien que j'ai fait: 24.392621835300936
fastdtw euclidien: 49.6623344800739
dtw cosine que j'ai fait: 0.5996959224483412
fastdtw cosine: 0.5996959224483412


Ici la fonction matrice prend en entrée une liste de séries temporelles, et fait une matrice de similarité en donnant la distance dtw entre chaque paire de séries temporelles.
Peut être pratique pour faire du clustering. 

In [4]:
def matrice_dtw(X, distance):
  N=len(X) #nb de pixels dans une image, = nb de séries temporelles qu'on a. chaque série = T vecteurs de 4 bandes
  R=np.zeros((N,N))
  for i in range(N):
    for j in range(N):
        R[i,j]= dtw.fastdtw(X[i],X[j],dist=distance)[0]
  return R

In [5]:
#pour faire une matrice de similarité : donne la similarité entre toutes les séries temporelles. la matrice X
def matrice_dtw_euclidien(X):
  N=len(X) #nb de pixels dans une image, = nb de séries temporelles qu'on a. chaque série = T vecteurs de 4 bandes
  R=np.zeros((N,N))
  for i in range(N):
    for j in range(N):
        R[i,j]=dtw_euclidien(X[i],X[j])
  return R 


#pour tester avec le cosin
def matrice_dtw_cosine(X):
  N=len(X) #nb de pixels dans une image, = nb de séries temporelles qu'on a. chaque série = T vecteurs de 4 bandes
  R=np.zeros((N,N))
  for i in range(N):
    for j in range(N):
        R[i,j]=dtw_cosine(X[i],X[j])
  return R 

print(matrice_dtw_euclidien(data))
print(matrice_dtw_cosine(data))


[[ 0.         24.20743687 28.87905816]
 [24.20743687  0.         24.39262184]
 [28.87905816 24.39262184  0.        ]]
[[0.         0.76730468 0.88925883]
 [0.76730468 0.         0.59969592]
 [0.88925883 0.59969592 0.        ]]


***KMEANS***

In [6]:
print(kmeans_dtw(data,3,50,euclidean)) # kmeans avec un dtw euclidien
print(kmeans_dtw(data,3,50,cosine)) # kmeans avec un dtw cosine

AttributeError: 'function' object has no attribute 'cdist'

In [ ]:
# # Function to implement steps given in the previous section - version pour dtw
# def kmeans_dtw(x, k, no_of_iterations,distance):
#     idx = np.random.choice(len(x), k, replace=False)
#     #print(x)

#     # Randomly choosing Centroids 
#     centroids = x[idx, :]  # Step 1
#     #centroids=centroids.reshape((3,6,1))

#     for _ in range(no_of_iterations):
#         # Finding the distance between centroids and all the data points
#         distances = np.zeros((len(x), k))
#         for i in range(len(x)):
#             for j in range(len(centroids)):
#                 distances[i, j] = dtw.fastdtw(x[i], centroids[j],dist=distance)[0]  # Step 2
#         #print("distances:",distances)

#         # Centroid with the minimum Distance
#         points = np.array([np.argmin(i) for i in distances])  # Step 3, on a le numéro de centroid de chaque point
#         #print("points:",points)
#         # Updating Centroids by taking the mean of the Cluster it belongs to
#         centroids = np.array([x[points == idx].mean(axis=0) for idx in range(k)])  # Updated Centroids, on fait la moyenne

#     # for i in range(0,k):
#     #     print("Cluster",i,":")
#     #     for j in points:
#     #         if (j==i):
#     #             print(x[j])
                

#     return points


# print(kmeans_dtw(data,3,50,euclidean)) # kmeans avec un dtw euclidien
# print(kmeans_dtw(data,3,50,cosine)) # kmeans avec un dtw cosine

[2 0 1]
[1 0 2]


***DBSCAN***


In [ ]:
data = np.array(data)

In [ ]:
from sklearn.cluster import DBSCAN
#matrice de similarité
print(matrice_dtw(data, euclidean))

#il faut bien choisir le paramètre eps
clustering_dbscan_euc = DBSCAN(eps=50,min_samples=5, metric='precomputed').fit(matrice_dtw_euclidien(data))
labels_dbscan_euc = clustering_dbscan_euc.labels_

print(labels_dbscan_euc)

[[ 0.         42.78197885 54.34716037]
 [42.78197885  0.         49.66233448]
 [54.34716037 49.66233448  0.        ]]
[-1 -1 -1]


In [ ]:
#matrice de similarité
print(matrice_dtw_cosine(data))

clustering_dbscan_cos = DBSCAN(eps=3, min_samples=5, metric='precomputed').fit(matrice_dtw_cosine(data))
labels_dbscan_cos = clustering_dbscan_cos.labels_

print(labels_dbscan_cos)

[[0.         0.76730468 0.88925883]
 [0.76730468 0.         0.59969592]
 [0.88925883 0.59969592 0.        ]]
[-1 -1 -1]


***OPTICS***


In [ ]:
from sklearn.cluster import OPTICS

clustering_optics_euc = OPTICS(min_samples=2, metric='precomputed').fit(matrice_dtw_euclidien(data))
labels_optics_euc = clustering_optics_euc.labels_

print(labels_optics_euc)


[0 0 0]


In [ ]:
clustering_optics_cos = OPTICS(min_samples=2, metric='precomputed').fit(matrice_dtw_cosine(data)) #le pb vient du reshaped
labels_optics_cos = clustering_optics_cos.labels_

print(labels_optics_cos)

[0 0 0]


***CAH***

In [ ]:
from sklearn.cluster import AgglomerativeClustering

clustering_CAH_euc = AgglomerativeClustering(metric='precomputed', linkage='complete').fit(matrice_dtw_euclidien(data))
labels_CAH_euc = clustering_CAH_euc.labels_
print(labels_CAH_euc)

[0 0 1]


In [ ]:
clustering_CAH_cos = AgglomerativeClustering(metric='precomputed', linkage='complete').fit(matrice_dtw_cosine(data))
labels_CAH_cos = clustering_CAH_cos.labels_
print(labels)

[1 0 0]


***TEST SUR LE SET DE PIXELS EXTRAITS DES IMAGES SATELLITES RÉELLES***

In [ ]:
import os
import cv2
from PIL import Image
import rasterio as rio
import re
import matplotlib.pyplot as plt


In [ ]:
pixels_de_interet = create_dic_pixels()

pix_danone= pixels_de_interet['pix_danone']
pix_agri = pixels_de_interet['pix_agri']
pix_ensta = pixels_de_interet['pix_ensta']
pix_apt = pixels_de_interet['pix_apt']
pix_lac = pixels_de_interet['pix_lac']
pix_foret =  pixels_de_interet['pix_foret']

In [ ]:
# # List of target pixels
# pixels_de_interet = pix_danone + pix_agri + pix_ensta + pix_apt + pix_lac + pix_foret
# len(pixels_de_interet)

54

In [ ]:
dir = "../../ressources/images"
images_list = os.listdir(dir)

images_2A = list()

prog = re.compile(r'\w+2A')
for image in images_list:
    if prog.match(image):
        images_2A.append(image)

        
images_2A.sort()
images_2A = np.array(sorted(images_2A, key=lambda date: date[16:24]))  

Help je sais pas utiliser raster + ça marche pas sur mon envt filrouge.

In [ ]:

list_to_use = images_2A
line = 0
matrice = np.zeros((len(pixels_de_interet), len(list_to_use)))
for image in list_to_use:
    with rio.open(dir+'/'+image, 'r') as ds:
        # arr = ds.read()
        band1 = ds.read(1)
        band2 = ds.read(2)
        band3 = ds.read(3)
        band4 = ds.read(4)
    # Extraction of target pixels
    for pixel,j in zip(pixels_de_interet, range(len(pixels_de_interet)-1)):
        pass
        matrice[j,line] = band3[pixel[0], pixel[1]]
    line +=1     
        
        

RasterioIOError: '../../ressources/images/crop_SENTINEL2A_20151226-111142-750_L2A_T31UDQ_D_V1-1.tif' not recognized as a supported file format.

In [ ]:
# chelou = np.argmax(matrice[0])
# matrice = np.delete(matrice,chelou,1)
type(matrice)
matrice.shape

(6, 10)

In [ ]:
#KMEANS - euclidien
print(kmeans_dtw(matrice,3,50,euclidean))

#DBSCAN - euclidien
clustering_dbscan = DBSCAN(eps=50,min_samples=5, metric='precomputed').fit(matrice_dtw_euclidien(matrice))
labels_dbscan = clustering_dbscan.labels_
print(labels_dbscan)

#OPTICS - euclidien
clustering_optics = OPTICS(min_samples=2, metric='precomputed').fit(matrice_dtw_euclidien(matrice))
labels_optics = clustering_optics.labels_
print(labels_optics)

#CAH - euclidien
clustering_CAH = AgglomerativeClustering(metric='precomputed', linkage='complete').fit(matrice_dtw_euclidien(matrice))
labels_CAH = clustering_CAH.labels_
print(labels_CAH)

ValueError: Input vector should be 1-D.

In [ ]:
#AFFICHAGE

import collections
def affichage(yhat,pix_interet):
    name = ''
    dico = collections.Counter(yhat)
    for key in list(dico.keys()):
        dico[key] = [f'number of vectors = {dico[key]}'] 
        for index,pos in zip(yhat,range(len(yhat))):
            if index == key:
                if 0<=pos<=8: name = 'pix_danone'
                elif 9<=pos<=17: name = 'pix_agri'
                elif 18<=pos<=26: name = 'pix_ensta'
                elif 27<=pos<=35: name = 'pix_apt'
                elif 36<=pos<=44: name = 'pix_lac'
                elif 45<=pos<=53: name = 'pix_apt'

                dico[key].append(f'{pix_interet[pos]}:{name}')

    for key in dico:
        print(f'cluster numero {key}:\n-------------------------------')
        for part in dico[key]:
            print(f'{part}')
        print('-------------------------------')


affichage(yhat_kmeans,pixels_de_interet)
affichage(yhat_dbscan,pixels_de_interet)
affichage(yhat_optics,pixels_de_interet)
affichage(yhat_cah,pixels_de_interet)


NameError: name 'matrice' is not defined